# The Swap Test Algorithm

The swap test is a quantum function that checks the overlap between two quantum states: the inputs of the function are two quantum registers of the same size, $|\psi_1\rangle, \,|\psi_2\rangle$, and it returns as an output a single "test" qubit whose state encodes the overlap between the two inputs: $|q\rangle_{\rm test} = \alpha|0\rangle + \sqrt{1-\alpha^2}|1\rangle$, with
$$
\alpha^2 = \frac{1}{2}\left(1+|\langle \psi_1 |\psi_2 \rangle |^2\right).
$$
Thus, the probability to measure the test qubit at state $0$ is 1 if the states are identical (up to a global phase), and 0 if the states are orthogonal to each other.

The quantum model starts with an H gate on the test qubit, followed by swapping between the two states controlled on the test qubit, and a final H gate on the test qubit:

<center>
<table><tr>
<td> <img src="https://classiq-docs-images.s3.amazonaws.com/swap_test_closed.png" style="width:100%">
<td> <img src="https://classiq-docs-images.s3.amazonaws.com/swap_test_opened.png" style="width:100%">
</tr></table>
<figcaption align = "middle"> Closed (left panel) and opened (right panel) visualization of the swap test algorithm </figcaption>
</center>

Prepare two random states:

In [12]:
import numpy as np

np.random.seed(12)

NUM_QUBITS = 3
amps1 = 1 - 2 * np.random.rand(2**NUM_QUBITS)
amps2 = 1 - 2 * np.random.rand(2**NUM_QUBITS)
amps1 = amps1 / np.linalg.norm(amps1)
amps2 = amps2 / np.linalg.norm(amps2)

Create a model and synthesize:

In [13]:
from classiq import (
    Output,
    QArray,
    QBit,
    QFunc,
    create_model,
    execute,
    prepare_amplitudes,
    set_execution_preferences,
    swap_test,
    synthesize,
)
from classiq.execution import ExecutionPreferences


@QFunc
def main(test: Output[QBit]):

    state1 = QArray("state1")
    state2 = QArray("state2")
    prepare_amplitudes(amps1.tolist(), 0.0, state1)
    prepare_amplitudes(amps2.tolist(), 0.0, state2)
    swap_test(state1, state2, test)


qmod = create_model(main)
qmod = set_execution_preferences(
    qmod, execution_preferences=ExecutionPreferences(num_shots=1e5)
)
qprog = synthesize(qmod)

In [14]:
from classiq import show

show(qprog)

In [5]:
with open("swap_test.qmod", "w") as f:
    f.write(qmod)

Verify the results:

In [6]:
res = execute(qprog).result()

In [7]:
overlap_from_swap_test = np.sqrt(
    2 * res[0].value.counts["0"] / sum(res[0].value.counts.values()) - 1
)
exact_overlap = np.abs(amps1 @ amps2)

In [8]:
print("States overlap from Swap-Test result:", overlap_from_swap_test)
print("States overlap from classical calculation:", exact_overlap)

States overlap from Swap-Test result: 0.4708078164176971
States overlap from classical calculation: 0.4697203723475908


In [10]:
RTOL = 0.05
assert np.isclose(
    overlap_from_swap_test, exact_overlap, RTOL
), f"""
The quantum result is too far than classical one, by a relative tolerance of {RTOL}. Please verify your parameters"""